In [1]:
import numpy as np
import reader

In [2]:
# List of stopwords
stopwords = set(["a","about","above","after","again","against","all","am","an","and","any","are","aren","'t","as","at","be","because","been","before","being","below","between","both","but","by","can","cannot","could","couldn","did","didn","do","does","doesn","doing","don","down","during","each","few","for","from","further","had","hadn","has","hasn","have","haven","having","he","he","'d","he","'ll","he","'s","her","here","here","hers","herself","him","himself","his","how","how","i","'m","'ve","if","in","into","is","isn","it","its","itself","let","'s","me","more","most","mustn","my","myself","no","nor","not","of","off","on","once","only","or","other","ought","our","ours","ourselves","out","over","own","same","shan","she","she","'d","she","ll","she","should","shouldn","so","some","such","than","that","that","the","their","theirs","them","themselves","then","there","there","these","they","they","they","they","'re","they","this","those","through","to","too","under","until","up","very","was","wasn","we","we","we","we","we","'ve","were","weren","what","what","when","when","where","where","which","while","who","who","whom","why","why","with","won","would","wouldn","you","your","yours","yourself","yourselves"])


In [31]:
dataset = "Enron6"

## Read in  spam and ham data

In [32]:
def load_data(trainingdir, stemming=False, lowercase=False, silently=False):
    print(f"Stemming is {stemming}")
    print(f"Lowercase is {lowercase}")
    train_set, train_labels = reader.load_dataset_main(trainingdir,stemming,lowercase,silently)
    return train_set, train_labels

In [33]:
training_dir = "data/"+dataset

In [34]:
train_set, train_labels = load_data(training_dir)

Stemming is False
Lowercase is False


In [35]:
len(train_set)
len(train_labels)

6000

## Use TF-IDF

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = []
for lst in train_set:
    corpus.append(' '.join(lst))


In [37]:
vectorizer = TfidfVectorizer(stop_words=stopwords)
tf_idf_score = vectorizer.fit_transform(corpus)
print(tf_idf_score.shape)

(6000, 70885)


In [38]:
from sklearn.preprocessing import normalize

normalized_matrix = normalize(tf_idf_score, norm='l1', axis=1)

print(normalized_matrix.shape)

(6000, 70885)


In [39]:
# Stores all th enormalized tf-idf scores
normalized_tf_idf = np.sum(normalized_matrix, axis=0)

print(normalized_tf_idf.shape)

(1, 70885)


In [40]:
words = vectorizer.get_feature_names_out()
print(words.shape)

(70885,)


## Save words and tf-idf in json

In [41]:
import json

# Step 1: Convert lists to a dictionary

word_freq_dict = {words[i]:normalized_tf_idf[0,i] for i in range(len(words))}

print(word_freq_dict)
# Step 2: Save the dictionary to a JSON file
output_file_path = "preprocessed_data/"+dataset +"_word_tf-idf.json"

with open(output_file_path, "w") as json_file:
    json.dump(word_freq_dict, json_file)

print(f"The dictionary has been saved to {output_file_path}")

{'00': 6.36376866138666, '000': 7.584951797673369, '0000': 0.050119214423653816, '000000': 0.01775184127309683, '00000000': 0.010091356290830349, '000000001606': 0.022164314853076652, '00000004150000000192': 0.046634117945460754, '0001': 0.013174636190461601, '0006509': 0.014324871439415354, '0006510': 0.014324871439415354, '000750013': 0.011648558238428978, '0007571982': 0.013179931205449035, '00098': 0.010235850090503257, '001': 0.19711879628795037, '00101': 0.017453526624818593, '0012': 0.0025126371061669935, '00153': 0.013023472398531804, '0016': 0.014246327786738733, '0017': 0.019939133372102194, '0018': 0.01134435035396113, '0018457': 0.012055834938333856, '002': 0.11285069966324576, '0020': 0.012345685708542838, '0021': 0.032907104131608786, '00218': 0.017023679941515008, '0022': 0.0842834851982526, '00221': 0.016671593349741144, '0022505287290': 0.00942845436675052, '00229': 0.01860066561347477, '00233244774480': 0.011721352600085971, '0025': 0.01005897322759816, '0026654': 0.0

## Select the top N words with highest tf-idf scores

In [42]:
# Combine words and frequencies into a list of tuples
word_frequency_pairs = [(words[i],normalized_tf_idf[0,i]) for i in range(len(words))]

# Sort the list based on frequencies in descending order
sorted_word_frequency_pairs = sorted(word_frequency_pairs, key=lambda x: x[1], reverse=True)

# Set the value of N (top N words)
N = 200

# Extract the top N words and their frequencies
top_N_words = sorted_word_frequency_pairs[:N]

print(top_N_words[:20])

[('subject', 27.028131749455014), ('enron', 18.13457971162684), ('com', 17.082870045045205), ('http', 17.081412581113504), ('will', 15.10436478522517), ('please', 14.82511332204877), ('get', 11.12665269294277), ('now', 10.376964421832014), ('www', 9.585121864772121), ('tw', 8.98503363438492), ('online', 8.89521284311577), ('re', 8.889841180281836), ('utf', 8.525358464361132), ('us', 8.359823149686521), ('new', 8.350537409374079), ('time', 7.982830533883808), ('email', 7.958656810042825), ('want', 7.756540015731299), ('attached', 7.740427532605987), ('company', 7.711001239979415)]


## Count the frequency of each word

In [43]:
ham_dict = {}
spam_dict = {}

# ham is 1; sam is 0;
for i in range(len(train_set)):
    for word in words:
        if word in train_set[i]:
            if train_labels[i] == 1:
                if not word in ham_dict.keys():
                    ham_dict[word] = 0
                ham_dict[word] += 1
            else:
                if not word in spam_dict.keys():
                    spam_dict[word] = 0
                spam_dict[word] += 1

    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900


In [44]:
output_file_path = "preprocessed_data/"+dataset +"_ham_freq.json"

with open(output_file_path, "w") as json_file:
    json.dump(ham_dict, json_file)

print(f"Ham dictionary has been saved to {output_file_path}")


output_file_path = "preprocessed_data/"+dataset +"_spam_freq.json"

with open(output_file_path, "w") as json_file:
    json.dump(spam_dict, json_file)

print(f"Spam dictionary has been saved to {output_file_path}")

Ham dictionary has been saved to preprocessed_data/Enron6_ham_freq.json
Spam dictionary has been saved to preprocessed_data/Enron6_spam_freq.json
